Necessary Imports

In [85]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Flatten, Dense, MaxPooling2D, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy

Custom Layer for weights and bias

In [90]:
class CustomConv2D(Layer):
    def __init__(self, filters, kernel_size, activation='relu'):
        super(CustomConv2D, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.activation = tf.keras.activations.get(activation)

# kernal - filter, filter - number of kernal
    def build(self, input_shape):
        self.kernel = self.add_weight(
            
            shape=(self.kernel_size[0], self.kernel_size[1], input_shape[-1], self.filters),
            initializer='glorot_uniform',
            trainable=True,
            name='kernel'

        )

        self.bias = self.add_weight(
            shape=(self.filters,),
            initializer='zeros',
            trainable=True,
            name='bias'
        )

    def call(self, inputs):
        output = tf.nn.conv2d(inputs, self.kernel, strides=[1, 1, 1, 1], padding='SAME')
        output = tf.nn.bias_add(output, self.bias)
        output = self.activation(output)
        return output


custom Dense Layer

In [91]:
class CustomDense(Layer):
    def __init__(self, units, activation='relu'):
        super(CustomDense, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        self.weight = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='glorot_uniform',  
            trainable=True,
            name='weights'
        )

        self.bias = self.add_weight(
            shape=(self.units,),
            initializer='zeros',  
            trainable=True,
            name='bias'
        )

    def call(self, inputs):
        output = tf.matmul(inputs, self.weight)
        output = tf.nn.bias_add(output, self.bias)
        output = self.activation(output)
        return output


CustomModel

In [92]:
class CustomModel(Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = CustomConv2D(3, (3,3))
        self.pool1 = MaxPooling2D((2,2))
        self.conv2 = CustomConv2D(6, (6,6))
        self.pool2 = MaxPooling2D((2,2))  
        self.flatten = Flatten()
        self.dense1  = CustomDense(64)
        self.dense2 = CustomDense(32)
        self.dense3 = CustomDense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)  # Corrected call to pool2
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        output = self.dense3(x)
        return output


In [93]:
model = CustomModel()  # Create an instance of your custom model
# Call the model on a batch of data to build it implicitly
# For example, you can create a dummy input tensor
dummy_input = tf.zeros((1, 32, 32, 3))  # Assuming input shape is (batch_size, height, width, channels)
model(dummy_input)
# Now, you can access the model summary
model.summary()


Model: "custom_model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 custom_conv2d_32 (CustomCo  multiple                  84        
 nv2D)                                                           
                                                                 
 max_pooling2d_16 (MaxPooli  multiple                  0         
 ng2D)                                                           
                                                                 
 custom_conv2d_33 (CustomCo  multiple                  654       
 nv2D)                                                           
                                                                 
 max_pooling2d_17 (MaxPooli  multiple                  0         
 ng2D)                                                           
                                                                 
 flatten_16 (Flatten)        multiple              